# 第4章 現象を説明・予測する統計モデルを作りたい(2) <br> ----階層的重回帰分析

階層的重回帰分析って初めて聞く。なんだろうか楽しみだな。

In [2]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler
import scipy.stats as st

import statsmodels.api as sm
import statsmodels.formula.api as sfm

from statsmodels.formula.api import ols

## データと手法の概要

### データの概要

### 分析の目的と概要 

## 階層的重回帰分析 

階層的重回帰分析は変数の数変えて比較するみたいな感じか。\
ただ定量的に比較できるとうれしい。

### モデル作成 

### 階層的重回帰分析の実行 

In [3]:
df_sts = pd.read_csv("../data/第04章/ストレス.csv", 
                     encoding="shift-jis")

In [4]:
df_sts.head()

,ストレス,サポート,バーンアウト1,バーンアウト2
0,2.9,3.3,2.2,2.3
1,3.1,3.6,2.7,3.0
2,2.3,3.5,3.1,3.3
3,3.7,3.2,3.4,2.7
4,3.7,3.3,3.7,3.5


In [5]:
model1 = ols("バーンアウト2~バーンアウト1", data=df_sts)
res1 = model1.fit()

res1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                バーンアウト2   R-squared:                       0.404
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                     202.0
Date:                Fri, 20 Mar 2020   Prob (F-statistic):           2.38e-35
Time:                        20:44:32   Log-Likelihood:                -106.66
No. Observations:                 300   AIC:                             217.3
Df Residuals:                     298   BIC:                             224.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0762      0.118      9.087      0.000       0.843       1.309
バーンアウト1        0.6125      0.043     14.211      0.000       0.528       0.697
==============================================================================
Omnibus:                       24.543   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.624
Skew:                          -0.307   Prob(JB):                     1.25e-15
Kurtosis:                       5.261   Cond. No.                         18.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
model2 = ols("バーンアウト2~バーンアウト1+ストレス+サポート", data=df_sts)
res2 = model2.fit()

res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                バーンアウト2   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                     70.52
Date:                Fri, 20 Mar 2020   Prob (F-statistic):           1.95e-34
Time:                        20:44:32   Log-Likelihood:                -103.39
No. Observations:                 300   AIC:                             214.8
Df Residuals:                     296   BIC:                             229.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1923      0.209      5.704      0.000       0.781       1.604
バーンアウト1        0.5801      0.045     12.989      0.000       0.492       0.668
ストレス           0.0742      0.035      2.149      0.032       0.006       0.142
サポート          -0.0809      0.048     -1.696      0.091      -0.175       0.013
==============================================================================
Omnibus:                       22.823   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               69.813
Skew:                          -0.219   Prob(JB):                     6.92e-16
Kurtosis:                       5.322   Cond. No.                         57.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

二回重回帰分析をすることで、その係数や、R2の変化をみてそれぞれの回帰が説明できているかを考えてる\
こんな時は見てみよう

### 決定係数の増分に関する検定 

In [7]:
F_1 = (res2.rsquared - res1.rsquared ) / (res2.df_model-res1.df_model)
F_2 = (1-res2.rsquared) / (len(df_sts)-res2.df_model-1)
F_1 / F_2 

3.2641870958134342

本との差は使う数値の桁数によるものと考えられる。

In [8]:
F_12 = (round(res2.rsquared ,4)- round(res1.rsquared ,4)) / (res2.df_model-res1.df_model)
F_22 = (1-round(res2.rsquared ,4)) / (len(df_sts)-res2.df_model-1)
F_12 / F_22 

3.273662551440335

これで値がそろった。

In [9]:
1 - st.f.cdf(F_12 / F_22 , dfn = 2, dfd = 296)

0.03924300313771112

次のAnovaについての検定は上で作成したFについて検定するだけなので、省略。\
ただ結果を引数にできるのは、Rの便利さだな。

### AICとBICによるモデルの比較

In [10]:
res1.aic,  res2.aic

(217.3169588000103, 214.77227348143902)

In [11]:
res1.bic , res2.bic

(224.7245237493227, 229.58740338006382)

## 重回帰分析での交互作用効果の検討

### 交互作用効果を検討するためのモデル

### 中心化

In [12]:
df_sts["ストレス_中心化"] = df_sts["ストレス"]-df_sts["ストレス"].mean()
df_sts["サポート_中心化"] = df_sts["サポート"]-df_sts["サポート"].mean()

In [13]:
df_sts["交互作用"] = df_sts["ストレス"]*df_sts["サポート"]
df_sts["交互作用_中心化"] = df_sts["ストレス_中心化"]*df_sts["サポート_中心化"]

In [14]:
df_sts[["ストレス","サポート", "交互作用" ]].corr()

,ストレス,サポート,交互作用
ストレス,1.000000,0.118076,0.848102
サポート,0.118076,1.000000,0.609125
交互作用,0.848102,0.609125,1.000000


In [15]:
import matplotlib.pyplot as plt

In [16]:
df_sts[["ストレス_中心化","サポート_中心化", "交互作用_中心化" ]].corr()

,ストレス_中心化,サポート_中心化,交互作用_中心化
ストレス_中心化,1.000000,0.118076,-0.099772
サポート_中心化,0.118076,1.000000,-0.235947
交互作用_中心化,-0.099772,-0.235947,1.000000


### 交互作用効果の検討 

In [17]:
df_sts["バーンアウト1_中心化"] =df_sts["バーンアウト1"]-df_sts["バーンアウト1"].mean() 

In [18]:
model3 = ols("バーンアウト2 ~ バーンアウト1_中心化 + サポート_中心化 + ストレス_中心化+サポート_中心化*ストレス_中心化",
          data = df_sts)
res3 = model3.fit()
res3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                バーンアウト2   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     55.48
Date:                Fri, 20 Mar 2020   Prob (F-statistic):           7.55e-35
Time:                        20:44:33   Log-Likelihood:                -100.14
No. Observations:                 300   AIC:                             210.3
Df Residuals:                     295   BIC:                             228.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.7397      0.020    138.673      0.000       2.701       2.779
バーンアウト1_中心化           0.5715      0.044     12.878      0.000       0.484       0.659
サポート_中心化             -0.1101      0.049     -2.263      0.024      -0.206      -0.014
ストレス_中心化              0.0696      0.034      2.032      0.043       0.002       0.137
サポート_中心化:ストレス_中心化    -0.1556      0.061     -2.542      0.012      -0.276      -0.035
==============================================================================
Omnibus:                       23.121   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.951
Skew:                          -0.249   Prob(JB):                     1.76e-15
Kurtosis:                       5.278   Cond. No.                         3.26
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
def anova(res1, res2):
    F_12 = (res2.rsquared - res1.rsquared ) / (res2.df_model-res1.df_model)
    F_22 = (1-res2.rsquared ) / (len(df_sts)-res2.df_model-1)
    print(f"F statisitc :{F_12 / F_22}") 
    
    DFN = res2.df_model-res1.df_model
    DFD = len(df_sts)-res2.df_model-1
    pval = 1 - st.f.cdf(F_12 / F_22 , dfn = DFN, dfd = DFD)
    print(f"p_val       : {pval}")

In [20]:
anova(res2, res3)

F statisitc :6.460902479969797
p_val       : 0.011538977610857093


### 標準偏回帰係数の算出

In [21]:
from scipy import stats as st

In [22]:
df_z_sts = pd.DataFrame(st.zscore(df_sts, axis = 0), columns=df_sts.columns)

In [23]:
model3_z = ols("バーンアウト2 ~ バーンアウト1 + サポート + ストレス+サポート*ストレス",
          data = df_z_sts)
res3_z = model3_z.fit()
res3_z.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                バーンアウト2   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     55.48
Date:                Fri, 20 Mar 2020   Prob (F-statistic):           7.55e-35
Time:                        20:44:33   Log-Likelihood:                -341.55
No. Observations:                 300   AIC:                             693.1
Df Residuals:                     295   BIC:                             711.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0105      0.044      0.237      0.813      -0.076       0.097
バーンアウト1        0.5931      0.046     12.878      0.000       0.502       0.684
サポート          -0.1045      0.046     -2.263      0.024      -0.195      -0.014
ストレス           0.0934      0.046      2.032      0.043       0.003       0.184
サポート:ストレス     -0.0886      0.035     -2.542      0.012      -0.157      -0.020
==============================================================================
Omnibus:                       23.121   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.951
Skew:                          -0.249   Prob(JB):                     1.76e-15
Kurtosis:                       5.278   Cond. No.                         1.70
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 単純傾斜分析 

### 単純傾斜分析の方法

### 単純傾斜分析の実行

In [24]:
df_sts["サポート_h"] = df_sts["サポート_中心化"] - df_sts["サポート_中心化"].std(ddof=1) 

In [25]:
model3_h = ols("バーンアウト2 ~ バーンアウト1 + ストレス_中心化+ サポート_h +サポート_h*ストレス_中心化",
          data = df_sts)
res3_h = model3_h.fit()
res3_h.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                バーンアウト2   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     55.48
Date:                Fri, 20 Mar 2020   Prob (F-statistic):           7.55e-35
Time:                        20:44:33   Log-Likelihood:                -100.14
No. Observations:                 300   AIC:                             210.3
Df Residuals:                     295   BIC:                             228.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.1450      0.120      9.532      0.000       0.909       1.381
バーンアウト1             0.5715      0.044     12.878      0.000       0.484       0.659
ストレス_中心化            0.0035      0.044      0.079      0.937      -0.083       0.090
サポート_h             -0.1101      0.049     -2.263      0.024      -0.206      -0.014
サポート_h:ストレス_中心化    -0.1556      0.061     -2.542      0.012      -0.276      -0.035
==============================================================================
Omnibus:                       23.121   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.951
Skew:                          -0.249   Prob(JB):                     1.76e-15
Kurtosis:                       5.278   Cond. No.                         19.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
df_sts["サポート_l"] = df_sts["サポート_中心化"] + df_sts["サポート_中心化"].std(ddof=1) 

In [27]:
model3_l = ols("バーンアウト2 ~ バーンアウト1 + ストレス_中心化+ サポート_l +サポート_l*ストレス_中心化",
          data = df_sts)
res3_l = model3_h.fit()
res3_l.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                バーンアウト2   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     55.48
Date:                Fri, 20 Mar 2020   Prob (F-statistic):           7.55e-35
Time:                        20:44:33   Log-Likelihood:                -100.14
No. Observations:                 300   AIC:                             210.3
Df Residuals:                     295   BIC:                             228.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.1450      0.120      9.532      0.000       0.909       1.381
バーンアウト1             0.5715      0.044     12.878      0.000       0.484       0.659
ストレス_中心化            0.0035      0.044      0.079      0.937      -0.083       0.090
サポート_h             -0.1101      0.049     -2.263      0.024      -0.206      -0.014
サポート_h:ストレス_中心化    -0.1556      0.061     -2.542      0.012      -0.276      -0.035
==============================================================================
Omnibus:                       23.121   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.951
Skew:                          -0.249   Prob(JB):                     1.76e-15
Kurtosis:                       5.278   Cond. No.                         19.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 交互作用効果のグラフ

## 報告例

## 重回帰分析における変数選択

### 変数選択とは

### 変数の実行 

In [28]:
df_bsb = pd.read_csv("../data/第04章/野球.csv", encoding="shift-jis")

StepAICはpythonになかったが、以下のURlを参考にした。
https://future-chem.com/esol-reg-aic/#stepwise_regression

In [29]:
import statsmodels.formula.api as smf
import itertools

In [84]:
def stepAIC(target, df):
    import copy
    descriptors = list(df.columns[df.columns!=target])
    
    formula = '{} ~ '.format(target) + ' + '.join(descriptors)
    
    f_model = smf.ols(formula=formula, data=df).fit()
    best_aic = f_model.aic
    best_model = f_model
    
    while descriptors:
        desc_selected = ''
        flag = 0
        for desk in descriptors:
            used_desks = copy.deepcopy(descriptors)
            used_desks.remove(desk)
            formula = '{} ~ '.format(target) + ' + '.join( used_desks)
            model = smf.ols(formula=formula, data=df).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_model = model
                desc_selected = desk
                flag = 1
        if flag:
            descriptors.remove(desc_selected)
        else:
            break
    return best_model
 
model_a = stepAIC(target = "年俸",df = df_bsb)
print(model_a.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1657.8607    873.064      1.899      0.060     -67.714    3383.436
打数            15.2037      5.103      2.979      0.003       5.118      25.289
本塁打          415.4792     77.501      5.361      0.000     262.301     568.657
四球           107.1365     35.415      3.025      0.003      37.140     177.133
三振           -73.7160     22.589     -3.263      0.001    -118.362     -29.070
